# Bygging av TikTok-bot med Appium og Android-telefon

Delvis dokumentasjon av metode.  
*Henrik Bøe, 18.01.2023*

Her er python-kode for en forenklet versjon av TikTok-boten vi brukte i NRK-saken "TikToks muskelkraft".  
  
**Vi deler ikke all koden, fordi det da blir svært enkelt for TikTok å blokkere akkurat hva vi har gjort her. Dette er derfor heller en grov oppskrift som gir en pekepinn på hvordan boten fungerer, og noen av stegene er beskrevet med pseudo-kode.**

### Forutsetninger  
  
Løsningen vår er basert på å styre en Android-telefon i utviklermodus med automatiseringsløsningen Appium og driveren UiAutomator2.  
    
For at dette skal virke må følgende være på plass:  
* Installasjon av Appium-server. (https://appium.io/)
* Utviklermodus aktivert på Android-telefon og USB-debugging skrudd på

### Opprett class AppiumBot
  
Vi opprettet en class vi kalte AppiumBot, fordi vi da kunne lage flere bot-instanser med ulike interesser og egenskaper.

Vi har ikke inkludert kode for funksjonene find_username, find_text, get_screenshot, db_update_interested og db_update_not_interested i dette eksempelet.

In [ ]:
class AppiumBot:
    def __init__(self, number_of_tiktoks, db_index):
        self.number_of_tiktoks = number_of_tiktoks
        self.db_index = db_index

    name = "UnnamedBot"
    interest_user = []
    interest_text = []
    long_duration = [20.0, 30.0]

    def run(self):
        desired_caps = dict(
            automationName = "UiAutomator2",
            deviceName = "Android",
            platformName = "Android",
            appPackage = "com.zhiliaoapp.musically",
            appActivity = "com.ss.android.ugc.aweme.main.MainActivity",
            noReset = True
        )
        driver = webdriver.Remote('APPIUM_SERVER_URL', desired_caps)
        time.sleep(3)

        for i in range(0, self.number_of_tiktoks):
            # Sjekker at TikTok-appen er åpen, og ikke har krasjet
            if driver.current_activity == "com.ss.android.ugc.aweme.main.MainActivity":
                try:
                    
                    start = time.time()
                    interested = "false"
                    date_and_time = datetime.datetime.utcnow()

                    # Finner brukernavn
                    user = find_username(driver, start)

                    # Finner tekst
                    text = find_text(driver, start)

                    # Sjekker om brukernavn eller interesser finnes
                    if any(elem in text.lower() for elem in self.interest_text):
                        interested = "true"
                    elif user in self.interest_user:
                        interested = "true"

                    # Hvis boten har funnet video den liker
                    if interested == "true":
                        # Sett viewtime til 
                        viewtime = random.uniform(self.long_duration[0], self.long_duration[1])

                        # Ta skjermdump og crop
                        image_name_output = get_screenshot()

                        # Send til Azure API
                        image_data = get_local_image_data(image_name_output)
                        azure_tags = [tag.name for tag in image_data[0]]
                        azure_text = image_data[2]

                        paused = "false"
                        liked = "false"
                        # Pauser video i en fjerdedel av tilfellene
                        if random.randint(1, 4) > 3:
                            paused = "true"
                            driver.tap(["KOORDINATER"])
                            time.sleep(random.randint(1,4))
                            driver.tap(["KOORDINATER"])
                            time.sleep(1)

                        # Liker video i en åttendedel av tilfellene
                        if random.randint(1, 8) > 7:
                            try:
                                driver.find_element(AppiumBy.ACCESSIBILITY_ID, 'Likerklikk').click()
                                liked = "true"
                            except Exception as e:
                                print(f"Feil ved forsøk på å like post: {e}")

                        time.sleep(viewtime)
                        actual_viewtime = time.time() - start

                        # Oppdaterer database med informasjon fra video
                        db_update_interested(user, text, date_and_time, actual_viewtime, interested, azure_tags,
                                             azure_text, image_name_output, paused, liked, self.db_index)
                        
                    else:
                        actual_viewtime = time.time() - start
                        # Oppdaterer database med informasjon fra video
                        db_update_not_interested(user, text, date_and_time, actual_viewtime, interested, self.db_index)

                    # Sveiper til neste video
                    driver.flick("KOORDINATER")

                except Exception as e:
                    print(e)
                    driver.quit()
            else:
                print(f"Current activity: {driver.current_activity}")
                driver.quit()
        driver.quit()

### Opprett bot-instanser
  
Deretter kunne vi lage bot-instanser med egenskapene vi ønsket. Først og fremst en liste med tekst boten skulle være interessert i, og en liste med aktuelle brukernavn.
  
Hvis tekst fra listen eller ett av brukernavnene dukket opp, ville den se lenger på videoen.

In [ ]:
bot = AppiumBot(random.randint(100, 200), "muskelbot")
bot.name = "MuskelBot"
bot.long_duration = [40.0, 50.0]

bot.interest_text = {"nøkkelord1", "nøkkelord2", "nøkkelord3"}

bot.interest_user = {"Brukernavn1", "Brukernavn2", "Brukernavn3"}

bot.run()